In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import tensorflow_datasets as tfds

In [66]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [67]:
import numpy as np

# 0: Negative
# 1: Possitive

train_data, test_data = imdb["train"], imdb["test"]

In [68]:
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

# Load dữ liệu từ data.numpy
for s, l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

# Load dữ liệu từ data.numpy
for s, l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

In [69]:
train_sentences[1]

"b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'"

In [70]:
train_labels[1]

0

In [71]:
# Biến list thành một matrix

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [72]:
# Khai báo
vocab_size = 10000
embedding_dim = 64
max_length = 140

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [74]:
# tạo một bộ thư viện

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [75]:
# Đánh số thứ tự cho các từ theo số lần xuất hiện ít hay nhiều của từ đó trong dict
word_index = tokenizer.index_word

In [ ]:
# word_index

In [77]:
# texts_to_sequences: chuyển các từ thành một chuỗi - là thứ tự của từ đó trong từ điển đã built

train_sentences = tokenizer.texts_to_sequences(train_sentences)

In [78]:
# train_sentences

In [79]:
# Kiểm tra xem từ có trong bộ từ điển hay không.
new_sentences = [
                 "Oh my god, this is terrible",
                 "I don't like this movies",
                 "Today, the weather is beautiful in Hà Nội"]

In [80]:
new_sentences = tokenizer.texts_to_sequences(new_sentences)
new_sentences

# 1 - <OOV>: Là những từ không có trong từ điển.

[[474, 63, 548, 12, 7, 400],
 [11, 174, 39, 12, 101],
 [558, 2, 5870, 7, 310, 9, 1, 1]]

In [86]:
padded_train_sentences = pad_sequences(train_sentences, maxlen=max_length, padding="post", truncating="post")

In [87]:
# 25000 câu và độ dài 140
padded_train_sentences.shape

(25000, 140)

In [84]:
# Cách hiểu: 
# 1: Chuyển test_sentences-text sang sequences-integer
# 2: Kiểm tra xem test_sequences có trong dict hay không.

# Kiểm tra

# Bước 1: Chuyển từ text sang integer sequences
test_sentences = tokenizer.texts_to_sequences(test_sentences)

# Bước 2: padding test_sequences để được một matrix có độ dài giống với tập train
padded_test_sequences = pad_sequences(test_sentences, maxlen=max_length, padding="post", truncating="post")

In [85]:
padded_test_sequences.shape

(25000, 140)

In [88]:
# Bước chuẩn bị dữ liệu ở trên:

# Bước 1: Đưa dữ liệu về dạng dict - số lần xuất hiện của 1 từ nhiều nhất sẽ được đưa lên đầu
# Bước 2: Đưa dự liệu về dạng matrix có cùng số chiều (dài: số từ có trong dict, rộng: độ dài của 1 câu)

In [90]:
# Huấn luyện mô hình dự đoán cảm xúc

# Khởi tạo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

In [112]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 140, 64)           640000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 8960)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                89610     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 729,621
Trainable params: 729,621
Non-trainable params: 0
_________________________________________________________________


In [113]:
# Compile model với các thuật toán tối ưu
model.compile(optimizer="adam", metrics=["acc"], loss="binary_crossentropy")

In [114]:
model.fit(padded_train_sentences, train_labels, epochs=10, validation_data=[padded_test_sequences, test_labels])

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4474 - acc: 0.7741 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1297 - acc: 0.9566 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0150 - acc: 0.9978 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 5.8900e-04 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 2.9481e-04 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 1.6970e-04 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 8/

In [153]:
# Test bộ training

test_sen = ["I love this movies"]

test_sen_tk = tokenizer.texts_to_sequences(test_sen)
padded_test_sen = pad_sequences(test_sen_tk, maxlen=max_length, padding="post", truncating="post")

# Dự đoán class
class_num = model.predict_classes(padded_test_sen)

# Dự đoán %
percent = model.predict(padded_test_sen)

if class_num[0][0] == 1:
  print("[INFO] This {} sentence is Possitive.".format(test_sen))
else:
  print("[INFO] This {} sentence is Negative.".format(test_sen))

[INFO] This ['I love this movies'] sentence is Possitive.


In [129]:
# padded_test_sen

In [154]:
e = model.layers[0]

In [162]:
weights = e.get_weights()[0]

In [165]:
weights.shape

(10000, 64)

In [170]:
import io

out_v = io.open("vecs.tsv", "w", encoding="utf-8")
out_m = io.open("meta.tsv", "w", encoding="utf-8")

for idx in range(1, vocab_size):
  word = tokenizer.index_word[idx]
  vector = weights[idx]
  out_m.write(word + "\n")
  out_v.write("\t".join([str(x) for x in vector]) + "\n")

out_m.close()
out_v.close()

In [ ]:
# tokenizer.word_index